# 🌈 LuminaFlow SDK - 快速入门

> **Train once, survive the noise. Build for the speed of light.**

本教程将带你用 **5 分钟** 上手 LuminaFlow SDK，在 MNIST 数据集上训练一个能在光子芯片上运行的神经网络。

## 学习目标

1. 了解如何用 `OpticalLinear` 替换传统的 `nn.Linear`
2. 使用 `NoiseAwareTrainer` 进行噪声感知训练
3. 使用 `benchmark_robustness` 可视化模型的鲁棒性

让我们开始吧！🚀



In [ ]:
# 安装 LuminaFlow SDK（如果还没安装）
# !pip install lumina-flow

# 导入必要的库
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# 导入 LuminaFlow SDK
import lumina.nn as lnn
from lumina.optim import NoiseAwareTrainer
from lumina.viz import benchmark_robustness

print("✅ 所有库导入成功！")
print(f"PyTorch 版本: {torch.__version__}")
print(f"设备: {'CUDA' if torch.cuda.is_available() else 'CPU'}")



## 步骤 1: 准备数据

我们使用 MNIST 手写数字数据集。如果你没有下载过，代码会自动下载。


In [ ]:
# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST 的均值和标准差
])

# 加载训练集和测试集
train_dataset = datasets.MNIST(
    '../data', 
    train=True, 
    download=True, 
    transform=transform
)
test_dataset = datasets.MNIST(
    '../data', 
    train=False, 
    transform=transform
)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

print(f"✅ 数据加载完成！")
print(f"训练集大小: {len(train_dataset)}")
print(f"测试集大小: {len(test_dataset)}")

# 可视化一个样本
sample_image, sample_label = train_dataset[0]
print(f"\n样本标签: {sample_label}")
plt.figure(figsize=(3, 3))
plt.imshow(sample_image.squeeze(), cmap='gray')
plt.title(f'标签: {sample_label}')
plt.axis('off')
plt.show()



## 步骤 2: 构建模型

现在我们来构建一个简单的 MLP（多层感知机）。关键是用 `OpticalLinear` 替换传统的 `nn.Linear`。

### 传统写法 vs Lumina 写法

**传统写法：**
```python
self.fc1 = nn.Linear(784, 512)
self.fc2 = nn.Linear(512, 10)
```

**Lumina 写法：**
```python
self.fc1 = lnn.OpticalLinear(784, 512, hardware_profile='lumina_nano_v1')
self.fc2 = lnn.OpticalLinear(512, 10, hardware_profile='lumina_nano_v1')
```

`OpticalLinear` 会自动模拟：
- 15% 光路噪声
- 4-bit DAC/ADC 量化
- 温度漂移效应


In [ ]:
class LuminaNet(nn.Module):
    """使用 LuminaFlow SDK 构建的光子神经网络"""
    
    def __init__(self):
        super(LuminaNet, self).__init__()
        
        # 展平层
        self.flatten = nn.Flatten()
        
        # 第一层：784 (28x28) -> 512
        # 使用 OpticalLinear 模拟光子芯片的第一层
        self.fc1 = lnn.OpticalLinear(
            784, 512,
            hardware_profile='lumina_nano_v1',  # 15% 噪声，4-bit 精度
            enable_wdm=True  # 启用波分复用
        )
        
        # 激活函数
        self.relu = nn.ReLU()
        
        # 第二层：512 -> 10 (输出类别)
        self.fc2 = lnn.OpticalLinear(
            512, 10,
            hardware_profile='lumina_nano_v1',
            enable_wdm=True
        )
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 创建模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LuminaNet().to(device)

print("✅ 模型创建成功！")
print(f"模型参数数量: {sum(p.numel() for p in model.parameters()):,}")
print(f"\n模型结构:")
print(model)



## 步骤 3: 噪声感知训练（NAT）

这是 LuminaFlow SDK 的核心功能！`NoiseAwareTrainer` 会自动在训练过程中注入硬件噪声，让模型学会在噪声环境下工作。

### 为什么需要 NAT？

- **问题**：在 GPU 上训练到 95% 准确率的模型，部署到光子芯片后可能掉到 60%
- **原因**：光子芯片存在光路噪声、量化误差、温度漂移等物理缺陷
- **解决**：在训练阶段就注入噪声，让模型提前适应

### 使用方法

只需要一行代码：
```python
trainer = NoiseAwareTrainer(model, optimizer, robustness_target=0.98)
trainer.fit(train_loader, epochs=5)
```


In [ ]:
# 创建优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 创建 NAT 训练器
# robustness_target: 目标鲁棒性（0.98 表示在噪声环境下保持 98% 的准确率）
trainer = NoiseAwareTrainer(
    model=model,
    optimizer=optimizer,
    robustness_target=0.98,
    noise_schedule='linear',  # 噪声调度策略：线性增长
    max_noise_level=0.20,     # 最大噪声水平：20%
    device=device
)

print("✅ 训练器创建成功！")
print("开始训练...")

# 开始训练（这里只训练 3 个 epoch 作为演示，实际可以训练更多）
trainer.fit(train_loader, epochs=3, val_loader=test_loader, verbose=True)

print("\n🎉 训练完成！")



## 步骤 4: 测试模型鲁棒性

现在我们来测试模型在不同噪声水平下的表现。这是 LuminaFlow SDK 的另一个核心功能。

`benchmark_robustness` 会自动：
1. 在 0% - 30% 的噪声水平下测试模型
2. 生成漂亮的抗噪曲线图
3. 保存报告到文件


In [ ]:
# 测试模型鲁棒性
print("开始鲁棒性测试...")
noise_levels, accuracies = benchmark_robustness(
    model=model,
    test_loader=test_loader,
    noise_levels=[0.0, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
    device=device,
    save_path="robustness_report.png",
    title="LuminaNet 鲁棒性分析 (NAT 训练)"
)

print("\n📊 测试结果:")
print("-" * 50)
print(f"{'噪声水平':<15} {'准确率':<15}")
print("-" * 50)
for noise, acc in zip(noise_levels, accuracies):
    print(f"{noise*100:>6.0f}%{'':>8} {acc:>6.2f}%")
print("-" * 50)

# 显示图表
plt.figure(figsize=(10, 6))
plt.plot([n*100 for n in noise_levels], accuracies, marker='o', linewidth=2, markersize=8)
plt.xlabel('噪声水平 (%)', fontsize=12)
plt.ylabel('准确率 (%)', fontsize=12)
plt.title('LuminaNet 鲁棒性分析 (NAT 训练)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xlim(0, max(noise_levels)*100*1.1)
plt.ylim(0, 105)
plt.tight_layout()
plt.show()



## 步骤 5: 对比实验（可选）

让我们对比一下：
- **标准训练**：不使用 NAT，直接在 GPU 上训练
- **NAT 训练**：使用 LuminaFlow SDK 的噪声感知训练

看看 NAT 训练在噪声环境下能带来多少提升！


In [ ]:
# 创建标准训练的模型（使用普通 nn.Linear）
class StandardNet(nn.Module):
    def __init__(self):
        super(StandardNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 512)  # 普通线性层
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 10)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 训练标准模型
print("训练标准模型（无 NAT）...")
standard_model = StandardNet().to(device)
standard_optimizer = optim.Adam(standard_model.parameters(), lr=0.001)

# 简单训练循环
for epoch in range(3):
    standard_model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        standard_optimizer.zero_grad()
        output = standard_model(data)
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        standard_optimizer.step()
    print(f"Epoch {epoch+1}/3 完成")

print("\n测试标准模型在噪声下的表现...")
std_noise_levels, std_accuracies = benchmark_robustness(
    model=standard_model,
    test_loader=test_loader,
    device=device,
    save_path="robustness_report_standard.png",
    title="标准模型鲁棒性分析（无 NAT）"
)

# 对比结果
print("\n📊 对比结果:")
print("=" * 70)
print(f"{'噪声水平':<15} {'标准训练':<15} {'NAT 训练':<15} {'提升':<15}")
print("=" * 70)
for i, noise in enumerate(noise_levels):
    std_acc = std_accuracies[i] if i < len(std_accuracies) else 0
    nat_acc = accuracies[i] if i < len(accuracies) else 0
    improvement = nat_acc - std_acc
    print(f"{noise*100:>6.0f}%{'':>8} {std_acc:>6.2f}%{'':>8} {nat_acc:>6.2f}%{'':>8} {improvement:>+6.2f}%")
print("=" * 70)

# 绘制对比图
plt.figure(figsize=(12, 6))
plt.plot([n*100 for n in noise_levels], std_accuracies, marker='s', linewidth=2, 
         markersize=8, label='标准训练', color='#E63946', linestyle='--')
plt.plot([n*100 for n in noise_levels], accuracies, marker='o', linewidth=2, 
         markersize=8, label='NAT 训练', color='#2E86AB')
plt.xlabel('噪声水平 (%)', fontsize=12)
plt.ylabel('准确率 (%)', fontsize=12)
plt.title('标准训练 vs NAT 训练 - 鲁棒性对比', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim(0, max(noise_levels)*100*1.1)
plt.ylim(0, 105)
plt.tight_layout()
plt.show()

print("\n✅ 结论：NAT 训练显著提升了模型在噪声环境下的鲁棒性！")



## 🎉 恭喜！

你已经成功完成了 LuminaFlow SDK 的快速入门！

### 你学到了什么？

1. ✅ 如何使用 `OpticalLinear` 替换传统的 `nn.Linear`
2. ✅ 如何使用 `NoiseAwareTrainer` 进行噪声感知训练
3. ✅ 如何使用 `benchmark_robustness` 测试模型鲁棒性
4. ✅ NAT 训练如何提升模型在噪声环境下的表现

### 下一步

- 📖 阅读 [完整文档](README.md)
- 🔬 尝试不同的硬件配置预设（`lumina_nano_v1` vs `lumina_micro_v1`）
- 🎯 在 CIFAR-10 等更复杂的数据集上测试
- 🚀 探索 WDM（波分复用）通道映射功能

### 资源

- **GitHub**: https://github.com/luminaflow/lumina-flow
- **文档**: https://luminaflow.readthedocs.io
- **问题反馈**: https://github.com/luminaflow/lumina-flow/issues

---

**Train once, survive the noise. Build for the speed of light.** ⚡

